In [1]:
!pip install transformers 
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 36.4 MB/s 
     |████████████████████████████████| 182 kB 63.2 MB/s 
     |████████████████████████████████| 7.6 MB 66.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 452 kB 33.0 MB/s 
     |████████████████████████████████| 213 kB 74.1 MB/s 
     |████████████████████████████████| 132 kB 80.3 MB/s 
     |████████████████████████████████| 127 kB 78.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
!unzip /content/"ner_dataset geo.zip"

Archive:  /content/ner_dataset geo.zip
  inflating: ner.csv                 


In [22]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModel
from datasets import Dataset , DatasetDict
import torch
from datasets import concatenate_datasets ,load_from_disk
from torch import nn
from sklearn.metrics import classification_report , f1_score , accuracy_score
from transformers import AdamW
from tqdm.notebook import tqdm_notebook
import matplotlib.pyplot as plt 

In [4]:
data = pd.read_csv("/content/ner.csv")
data.head()

,Sentence #,Sentence,POS,Tag
0,Sentence: 1,Thousands of demonstrators have marched throug...,"['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP'...","['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', '..."
1,Sentence: 2,Families of soldiers killed in the conflict jo...,"['NNS', 'IN', 'NNS', 'VBN', 'IN', 'DT', 'NN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,Sentence: 3,They marched from the Houses of Parliament to ...,"['PRP', 'VBD', 'IN', 'DT', 'NNS', 'IN', 'NN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
3,Sentence: 4,"Police put the number of marchers at 10,000 wh...","['NNS', 'VBD', 'DT', 'NN', 'IN', 'NNS', 'IN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,Sentence: 5,The protest comes on the eve of the annual con...,"['DT', 'NN', 'VBZ', 'IN', 'DT', 'NN', 'IN', 'D...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."


In [5]:
#ss = "Ariel Ticona 's wife gave birth to their daughter , Esperanza  Spanish for Hope , on Tuesday at a hospital in Copiapo , near the mine ."
#data[data["Sentence"]==ss]
data = data.drop(labels=[39290,5699,28978,31426,8331,39286,47182])

In [6]:
data[data['Sentence'].isnull()]

,Sentence #,Sentence,POS,Tag


In [7]:
sentences = data["Sentence"]

In [8]:
tags = list(set([s.replace("]","").replace("[","").replace("'","").replace(" ","") for sents in  data["Tag"] for s in sents.split(",")]))
tags_to_index = dict(zip(tags,range(0,len(tags))))

In [9]:
X_data_train , X_data_test  = train_test_split(data,test_size=0.15,random_state=20)

In [10]:
def get_words_tags(data):
  words_sents = []
  tags_sents = []
  for i,j in data.iterrows():
    sent_tags = [s.replace("]","").replace("[","").replace("'","").replace(" ","") for s in  j["Tag"].split(",")]
    words_sent = j["Sentence"].split(" ")
    words_sents.append(words_sent)
    tags_sents.append(sent_tags)
  return words_sents , tags_sents
  

In [11]:
words_train , tags_train = get_words_tags(X_data_train)
words_test , tags_test = get_words_tags(X_data_test)

In [12]:
train_data_dict = {"text_sents" : X_data_train["Sentence"],
                   "tag_sents" : tags_train,
                   "words_sents" : words_train}
test_data_dict = {"text_sents" : X_data_test["Sentence"],
                  "tag_sents" : tags_test,
                  "words_sents" : words_test}

In [13]:
train_dataset = Dataset.from_dict(train_data_dict)
test_dataset = Dataset.from_dict(test_data_dict)

In [14]:
train_dataset

Dataset({
    features: ['text_sents', 'tag_sents', 'words_sents'],
    num_rows: 40759
})

In [15]:
def preprocess(record, tokenizer):
    encoded_inputs = tokenizer(
        record["text_sents"],
        truncation=True,
        max_length=len(tokenizer.tokenize(record["text_sents"])) + 2,
        padding="max_length",
        return_overflowing_tokens=False,
        return_offsets_mapping=False,
    )
    with torch.no_grad():
        out = model(
            input_ids=torch.Tensor(encoded_inputs["input_ids"])
            .unsqueeze(0)
            .to(int)
            .to(device),
            attention_mask=torch.Tensor(encoded_inputs["attention_mask"])
            .unsqueeze(0)
            .to(int)
            .to(device),
            token_type_ids=torch.Tensor(encoded_inputs["token_type_ids"])
            .unsqueeze(0)
            .to(int)
            .to(device),
        )
    vectors = torch.cat(out[2], 0)

    word_vectors = []
    real = [x for x in record["words_sents"]]
    c = 0
    cc = 0
    t = []
    s = []
    w_c = torch.zeros((13, 768)).to(device)
    for i, token in enumerate(tokenizer.tokenize(record["text_sents"])):
        if token == real[c]:
            c += 1
            word_vectors.append(vectors[:, i + 1].unsqueeze(0))
            s.append(token)
        else:
            w_c += vectors[:, i + 1]
            t.append(token.replace("#", ""))
            if "".join(t) == real[c]:
                c += 1
                s.append("".join(t))
                word_vectors.append(w_c.unsqueeze(0))
                t = []
                w_c = torch.zeros((13, 768)).to(device)
                cc = 0
    labels = [tags_to_index[x] for x in record["tag_sents"]]
    if len(word_vectors) != len(labels):
        print(tokenizer.tokenize(record["text_sents"]))
        print(real)
        print(s)
        print(record["text_sents"])
        print("error")
    return {"word_embeddings": torch.cat(word_vectors, 0), "labels": labels}

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
c_model = "bert-base-cased"

tokenizer = AutoTokenizer.from_pretrained(c_model)
model = AutoModel.from_pretrained(c_model, output_hidden_states=True)
model.to(device)
for param in model.parameters():
          param.requires_grad = False
model.eval();

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [27]:
dt = train_dataset.train_test_split(0.6)
d1 = (
    train_dataset.train_test_split(0.5)["train"]
    .train_test_split(0.5)["train"]
    .train_test_split(0.5)["train"]
)
d2 = (
    train_dataset.train_test_split(0.5)["train"]
    .train_test_split(0.5)["train"]
    .train_test_split(0.5)["test"]
)
d3 = (
    train_dataset.train_test_split(0.5)["train"]
    .train_test_split(0.5)["test"]
    .train_test_split(0.5)["train"]
)
d4 = (
    train_dataset.train_test_split(0.5)["train"]
    .train_test_split(0.5)["test"]
    .train_test_split(0.5)["test"]
)
d5 = (
    train_dataset.train_test_split(0.5)["test"]
    .train_test_split(0.5)["train"]
    .train_test_split(0.5)["train"]
)
d6 = (
    train_dataset.train_test_split(0.5)["test"]
    .train_test_split(0.5)["train"]
    .train_test_split(0.5)["test"]
)
d7 = (
    train_dataset.train_test_split(0.5)["test"]
    .train_test_split(0.5)["test"]
    .train_test_split(0.5)["train"]
)
d8 = (
    train_dataset.train_test_split(0.5)["test"]
    .train_test_split(0.5)["test"]
    .train_test_split(0.5)["test"]
)

d1 = d1.map(
    lambda x: preprocess(x, tokenizer),
    remove_columns=d1.column_names,
    keep_in_memory=False,
)
d2 = d2.map(
    lambda x: preprocess(x, tokenizer),
    remove_columns=d2.column_names,
    keep_in_memory=False,
)
d3 = d3.map(
    lambda x: preprocess(x, tokenizer),
    remove_columns=d3.column_names,
    keep_in_memory=False,
)
d4 = d4.map(
    lambda x: preprocess(x, tokenizer),
    remove_columns=d4.column_names,
    keep_in_memory=False,
)
d5 = d5.map(
    lambda x: preprocess(x, tokenizer),
    remove_columns=d5.column_names,
    keep_in_memory=False,
)
d6 = d6.map(
    lambda x: preprocess(x, tokenizer),
    remove_columns=d6.column_names,
    keep_in_memory=False,
)
d7 = d7.map(
    lambda x: preprocess(x, tokenizer),
    remove_columns=d7.column_names,
    keep_in_memory=False,
)
d8 = d8.map(
    lambda x: preprocess(x, tokenizer),
    remove_columns=d8.column_names,
    keep_in_memory=False,
)

train_dataset = concatenate_datasets([d1, d2, d3, d4, d5, d6, d7, d8])
train_dataset

DatasetDict({
    train: Dataset({
        features: ['text_sents', 'tag_sents', 'words_sents'],
        num_rows: 19181
    })
    test: Dataset({
        features: ['text_sents', 'tag_sents', 'words_sents'],
        num_rows: 19181
    })
})

In [17]:
#train_dataset = train_dataset.map(lambda x : preprocess(x , tokenizer) , remove_columns = train_dataset.column_names,keep_in_memory=False)
test_dataset = test_dataset.map(lambda x : preprocess( x , tokenizer ) , remove_columns=test_dataset.column_names)

  0%|          | 0/7193 [00:00<?, ?ex/s]

In [18]:
test_dataset.save_to_disk("test_test")

Saving the dataset (0/13 shards):   0%|          | 0/7193 [00:00<?, ? examples/s]

In [25]:
test_dataset = load_from_disk("test_test")

In [26]:
test_dataset

Dataset({
    features: ['word_embeddings', 'labels'],
    num_rows: 7193
})

#edge probing

In [27]:
def plot(train_hist,test_hist,lbl="acc"):
  plt.plot([i for i in range(len(train_hist))], train_hist, label = "train_"+lbl)
  plt.plot([i for i in range(len(test_hist))], test_hist, label = "test_"+lbl)
  plt.xlabel('epochs') 
  plt.ylabel(lbl) 
  plt.legend()
  plt.show()

In [28]:
class Edge_probing(nn.Module):
  def __init__(self):
     super(Edge_probing, self).__init__()
     self.fc = nn.Linear(768 , len(tags))
  
  def forward(self,x):
    return self.fc(x)

In [77]:
def train(
    model,
    optimizer,
    loss_fn,
    train_dataset,
    validation_dataset,
    lay,
    batch_size=3,
    epochs=30,
    device="cuda",
):
    model.to(device)
    best_loss = 100
    best_score = 0
    best_epoch = 0
    acc_train = []
    acc_test = []
    loss_train = []
    loss_test = []
    for epoch in range(epochs):
        training_loss = 0.0
        val_loss = 0.0
        model.train()

        t_labels = []
        t_preds = []
        for i in tqdm_notebook(np.arange(0, len(train_dataset), batch_size)):

            embeddings = torch.cat(
                train_dataset[i : i + batch_size]["word_embeddings"], 0
            )[:, lay].to(device)
            labels = torch.cat(train_dataset[i : i + batch_size]["labels"], 0).to(
                torch.long
            )
            preds = model(embeddings)
            t_labels += labels.numpy().tolist()

            optimizer.zero_grad()
            preds = model(embeddings)
            loss = loss_fn(preds, labels.to(device))
            loss.backward()
            optimizer.step()
            training_loss += loss.item()
            t_preds += preds.argmax(axis=1).cpu().numpy().tolist()

        training_loss = training_loss / len(train_dataset)

        train_accuracy = accuracy_score(t_labels, t_preds, normalize=True)

        valid_loss, valid_acc, valid_f1_wieghted = validation(
            model, validation_dataset, loss_fn, lay, False, device
        )

        loss_train.append(training_loss)
        loss_test.append(valid_loss)
        acc_train.append(train_accuracy)
        acc_test.append(valid_acc)
        print(
            "Epoch: {},training loss: {:.2f} , train accuracy: {:.2f} ,validation loss :{:.2f} , validation acc:{:.2f}".format(
                epoch, training_loss, train_accuracy, valid_loss, valid_acc
            )
        )

    plot(acc_train, acc_test, "acc")
    plot(loss_train, loss_test, "loss")
    validation(model, validation_dataset, loss_fn, lay, True, device)

In [78]:
def validation(model, val_dataset, loss_fn, lay, test=True, device="cuda"):
    val_loss = 0.0
    model.eval()
    num_words = 0
    tps = 0
    reals = []
    preds_list = []
    for i in np.arange(0, len(val_dataset), 3):
        embeddings = torch.cat(val_dataset[i : i + 3]["word_embeddings"], 0)[:, lay].to(
            device
        )
        labels = torch.cat(val_dataset[i : i + 3]["labels"], 0).to(torch.long)
        preds = model(embeddings)
        loss = loss_fn(preds, labels.to(device))
        val_loss += loss.item()
        reals += labels.numpy().tolist()
        preds_list += preds.argmax(axis=1).cpu().numpy().tolist()

    val_loss = val_loss / len(val_dataset)
    accuracy = accuracy_score(reals, preds_list, normalize=True)
    val_f1_wighted = f1_score(reals, preds_list, average="macro")
    if test:
        print(classification_report(reals, preds_list))
    return val_loss, accuracy, val_f1_wighted

In [73]:
test_dataset = test_dataset.shuffle(40)

In [79]:
batch_size = 10
lay = 1
train_dataset.set_format(type="torch")
test_dataset.set_format(type="torch")
loss_fn = nn.CrossEntropyLoss()
edge_model = Edge_probing()
optimizer = AdamW(edge_model.parameters(), lr=2e-5)
train(
    edge_model,
    optimizer,
    loss_fn,
    test_dataset,
    test_dataset,
    lay,
    epochs=5,
    device="cuda",
)

  0%|          | 0/2398 [00:00<?, ?it/s]

TypeError: ignored

In [65]:
len(test_dataset) % 5

3

In [ ]:
test_dataset[252:252+3]["word_embeddings"].tolist()

#SI Probing

In [29]:
test_dataset.set_format(type="torch")
t_lables = torch.cat(test_dataset["labels"], 0)
test_vectors = test_dataset[0]["word_embeddings"][:, 12]
for i in tqdm_notebook(range(1, len(test_dataset))):
    test_vectors = torch.cat((test_vectors, test_dataset[i]["word_embeddings"][:, 12]))
test_vectors_norm = (test_vectors - torch.min(test_vectors, 0)[0]) / (
    torch.max(test_vectors, 0)[0] - torch.min(test_vectors, 0)[0]
)

  0%|          | 0/7192 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
test_vectors_norm.to(device).shape

In [ ]:
def get_distance_Matrix(vectors):
  # X = vectors.clone()
  # M = torch.matmul(X,X.T).to(device)
  # d = torch.diag(M)
  # W = torch.cat([d.unsqueeze(0) for i in range(len(vectors))],0).to(device)
  # W = torch.Tensor(W).to(device)
  # D = W + W.T - (2*M)
  # D.fill_diagonal_(torch.inf)
  # return D
  return torch.cdist(vectors.to(device),vectors.to(device)).fill_diagonal_(torch.inf)

In [ ]:
k = 40
best_feature_set = []
for fk in range(1, k):

    scores = []
    for i in tqdm_notebook(range(len(test_vectors_norm[0]))):
        D = get_distance_Matrix(
            test_vectors_norm[:, torch.Tensor(best_feature_set + [i]).to(torch.long)]
        )
        idxs = torch.argmin(D, 0)
        score = torch.sum(t_lables[idxs.cpu()] == t_lables)
        scores.append(score)
    index_best_new_feture = torch.argmax(torch.Tensor(scores))
    print("best score is :", scores[index_best_new_feture])
    best_feature_set = best_feature_set + [index_best_new_feture.tolist()]